# Agent
- What is an Agent?
  - Agents execute actions based on LLM reasoning
  - They pass the output back to LLM to determine if next action is required
  - Or if its OK to finish
- Goal
  - Build an Agent that can interact with search engine
    - You can ask question to agent
    - Agent will call the search tool
    - You can have conversation with agent

In [1]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

In [2]:
# pip install langchain
# pip install -qU langchain-mistralai


# Search Engine
The search engine that offers good support for LLM is  [Tavily](https://tavily.com/).  
Get a TAVILY_API_KEY which will be needed for this project

In [ ]:
# !pip install tavily-python

In [2]:
os.environ["TAVILY_API_KEY"] = getpass.getpass()


In [3]:
os.environ["MISTRAL_API_KEY"] = getpass.getpass()



In [4]:
# !pip install langchain-chroma

In [9]:
os.environ['HF_TOKEN'] = getpass.getpass()

 ········


## Create Tools
The first tool that we need to create for this project is Tavily search engine tool

In [4]:
from langchain_community.tools.tavily_search import TavilySearchResults

tavily = TavilySearchResults(max_results=2)
search_results = tavily.invoke("What is the weather in Ahmedabad today?")
print(search_results)

[{'url': 'https://www.hindustantimes.com/cities/ahmedabad-weather-today-aqi-and-rain-forecast-updates-july-4-2024-101720056621176.html', 'content': 'July 6, 2024: 32.07 °C : Light rain: July 7, 2024: 33.54 °C : Light rain: July 8, 2024: 34.24 °C : Broken clouds: ... Ahmedabad weather update on July 04, 2024. Get World Cup ready with Crickit ...'}, {'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Ahmedabad', 'region': 'Gujarat', 'country': 'India', 'lat': 23.03, 'lon': 72.62, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1720087651, 'localtime': '2024-07-04 15:37'}, 'current': {'last_updated_epoch': 1720087200, 'last_updated': '2024-07-04 15:30', 'temp_c': 33.0, 'temp_f': 91.4, 'is_day': 1, 'condition': {'text': 'Mist', 'icon': '//cdn.weatherapi.com/weather/64x64/day/143.png', 'code': 1030}, 'wind_mph': 5.6, 'wind_kph': 9.0, 'wind_degree': 260, 'wind_dir': 'W', 'pressure_mb': 999.0, 'pressure_in': 29.5, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 63, 'cloud': 

In [5]:
tools = [tavily]

## Use LLM
In this project we are using MistralAI language model

In [6]:
from langchain_mistralai import ChatMistralAI

model = ChatMistralAI(model="mistral-large-latest")

Model will not use the tools by itself.  
We have to enable the model to use the tools by calling .bind_tools method of Model

In [7]:
model_with_tools = model.bind_tools(tools)

we can call the model now  
Frist try with simple message
The model will not call the tool as it will not be required because we are not sending any query

In [13]:
from langchain_core.messages import HumanMessage
response = model_with_tools.invoke([
    HumanMessage(content="Hi")
    ]
)

print(f"Content String: {response.content}")
print(f"Tool Calls: {response.tool_calls}")

Content String: Hello! How can I assist you today? If you have any questions or need information on current events, feel free to ask.
Tool Calls: []


In [14]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'weather in SF'}, 'id': 'jUlqkiy24'}]


In the above example you would notice that the model advises us to call a tool. This is a decision made by the model  
It has not called the tool yet. Its asking us to make a tool call  
In order to perform a tool call we need to build an agent

In [ ]:
# !pip install langgraph

## Create Agent
- Use LangGraph to create Agent
- High level API
- It also provides low level control
- create agent and init it will llm model and tools. create_react_agent will call model.bind_tools under the hood

In [10]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [18]:
# testing the agent with simple query. Expected output is that it will not call the tool
response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

response["messages"]

[HumanMessage(content='hi!', id='120c99f7-cdc5-4a0a-bf2c-4e2eb53e6734'),
 AIMessage(content="Hello! How can I assist you today? If you have any questions or need information on current events, feel free to ask. I'm here to help you find comprehensive, accurate, and trusted results.", response_metadata={'token_usage': {'prompt_tokens': 109, 'total_tokens': 151, 'completion_tokens': 42}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-a7632074-6c92-4b2d-ba2a-b309d3482bdc-0', usage_metadata={'input_tokens': 109, 'output_tokens': 42, 'total_tokens': 151})]

In [19]:
# in this example it should invoke the tool
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
)
response["messages"]

[HumanMessage(content='whats the weather in sf?', id='f5c95724-87c5-48ce-94a5-be067eeb75f5'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '4qGAyjKZh', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in sf"}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 114, 'total_tokens': 144, 'completion_tokens': 30}, 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-45836a74-fa21-4cf9-bcda-b17944975abb-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': '4qGAyjKZh'}], usage_metadata={'input_tokens': 114, 'output_tokens': 30, 'total_tokens': 144}),
 ToolMessage(content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1720003064, \'localtime\': \'2024-07-03 3:3

As we can see from the above example that the agent has called the tool as it was instructed by the Model  
The output from the tool is fed back to the model which eventually gives output to Human

### Streaming Messages
- The agent performs multiple steps before it could generate the output
- It calles the invoke method and hence again the stream method is available to streaming out the messages


In [20]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'JkS35u1Ud', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "weather in sf"}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 114, 'total_tokens': 144, 'completion_tokens': 30}, 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-3f60f421-4e34-4269-8ca3-a763b647011a-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in sf'}, 'id': 'JkS35u1Ud'}], usage_metadata={'input_tokens': 114, 'output_tokens': 30, 'total_tokens': 144})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.accuweather.com/en/us/san-francisco/94103/march-weather/347629", "content": "Get the monthly weather forecast for San Francisco, CA, including daily high/low, historical averages, to help you plan ahead."}, {"url": "https://weatherspark.com/h/m/557/2024/3/Historical-Weather-in-March-2024-in-San-Francisco-California-Unit

### Streaming Tokens
Even tokens could be streamed  
for this the stream_events method could be invoked.  
The method streams all events  

In [21]:
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

/home/ritesh/langchain_test/lib/python3.10/site-packages/langchain_core/_api/beta_decorator.py:87: LangChainBetaWarning: This API is in beta and may change in the future.
  warn_beta(


--
Starting tool: tavily_search_results_json with inputs: {'query': 'weather in sf'}
Done tool: tavily_search_results_json
Tool output was: [{'url': 'https://weatherspark.com/h/m/557/2024/3/Historical-Weather-in-March-2024-in-San-Francisco-California-United-States', 'content': 'San Francisco Temperature History March 2024. The daily range of reported temperatures (gray bars) and 24-hour highs (red ticks) and lows (blue ticks), placed over the daily average high (faint red line) and low (faint blue line) temperature, with 25th to 75th and 10th to 90th percentile bands.'}, {'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1720004207, 'localtime': '2024-07-03 3:56'}, 'current': {'last_updated_epoch': 1720003500, 'last_updated': '2024-07-03 03:45', 'temp_c': 20.6, 'temp_f': 69.1, 'is_day': 0, 'condition': {'t

## Adding Memory
- The agent is stateless. It does not remember history
- We nned to pass a checkpointer and thread_id to give it memory specific to a thread

In [8]:
from langgraph.checkpoint.sqlite import SqliteSaver

memory = SqliteSaver.from_conn_string(":memory:")    # in memeory storage

In [11]:
#give a checkpointer = memory to the agent
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [13]:
#stream messages
from langchain_core.messages import HumanMessage
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im Karishma!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Hello Karishma! How can I assist you today?', response_metadata={'token_usage': {'prompt_tokens': 113, 'total_tokens': 125, 'completion_tokens': 12}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-006e49ac-af44-44e8-bbad-a6bc5c65a5a2-0', usage_metadata={'input_tokens': 113, 'output_tokens': 12, 'total_tokens': 125})]}}
----


In [14]:
# now the agent will remember the previous conversation
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Based on our conversation, your name is Karishma.', response_metadata={'token_usage': {'prompt_tokens': 133, 'total_tokens': 145, 'completion_tokens': 12}, 'model': 'mistral-large-latest', 'finish_reason': 'stop'}, id='run-b34ddb4c-7dcc-4faf-9c9c-6330b68bafda-0', usage_metadata={'input_tokens': 133, 'output_tokens': 12, 'total_tokens': 145})]}}
----


In [15]:
# now we can make it call a tool
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="What is the latest news about Hcl Technologies stock symbol HCLTECH")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '1wrBxajQf', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "HCLTECH stock symbol"}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 164, 'total_tokens': 196, 'completion_tokens': 32}, 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-826a5671-184d-40f0-9d50-05ce708cf60b-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'HCLTECH stock symbol'}, 'id': '1wrBxajQf'}], usage_metadata={'input_tokens': 164, 'output_tokens': 32, 'total_tokens': 196})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://finance.yahoo.com/quote/HCLTECH.NS/", "content": "Find the latest HCL Technologies Limited (HCLTECH.NS) stock quote, history, news and other vital information to help you with your stock trading and investing."}, {"url": "https://www.google.com/finance/quote/HCLTECH:NSE", "content": "Get the latest HCL Technol

In [18]:
# now we can make it call a tool
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="Find the latest news and current stock price of HCLTECH. Summarize the same")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'pt7w8BHXl', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query": "latest news of HCLTECH and current stock price"}'}}]}, response_metadata={'token_usage': {'prompt_tokens': 1866, 'total_tokens': 1903, 'completion_tokens': 37}, 'model': 'mistral-large-latest', 'finish_reason': 'tool_calls'}, id='run-410d76df-1114-4d3c-b761-78fed2acbd36-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'latest news of HCLTECH and current stock price'}, 'id': 'pt7w8BHXl'}], usage_metadata={'input_tokens': 1866, 'output_tokens': 37, 'total_tokens': 1903})]}}
----
{'tools': {'messages': [ToolMessage(content='[{"url": "https://www.google.com/finance/quote/HCLTECH:NSE", "content": "Get the latest HCL Technologies Ltd (HCLTECH) real-time quote, historical performance, charts, and other financial information to help you make more informed trading and investment decisions."}, {"url